In [32]:
fs = require('fs');
dsv = require('d3-dsv');
simpleStats = require('simple-statistics');
_ = require('lodash');

''

''

In [33]:
ct_census = dsv.csvParse(fs.readFileSync('too_big/ss14hct.csv', 'utf8'));
// from http://www2.census.gov/programs-surveys/acs/data/pums/2014/1-Year/csv_hct.zip

''

''

In [34]:
// these PUMA IDs represent the Bridgeport-Stamford-Norwalk, CT (Metro) according to geocorr
bridgeport_2014 = ct_census.filter(d => +d.PUMA >= 100 && +d.PUMA <= 105);

/*
"ADJINC
Adjustment factor for income and earnings dollar amounts(6 implied decimal places)
1008425 .2014 factor (1.008425)

Note: The value of ADJINC inflation-adjusts reported income to 2014 dollars.
ADJINC applies to variables FINCP and HINCP in the housing record, and variables
INTP, OIP, PAP, PERNP, PINCP, RETP, SEMP, SSIP, SSP, and WAGP in the person record. 

HINCP
Household income (past 12 months)
bbbbbbbbb .N/A(GQ/vacant)
000000000 .No household income
-00059999 .Loss of -$59,999 or more
-00000001..-00059998 .Loss of $1 to -$59,998
000000001 .$1 or Break even
000000002..999999999 .Total household income in dollars (Components are rounded)

Note: Use ADJINC to adjust HINCP to constant dollars."

https://www2.census.gov/programs-surveys/acs/tech_docs/pums/data_dict/PUMSDataDict14.txt
*/

bridgeport_2014 = bridgeport_2014.filter(d => d.HINCP !== ' ')
bridgeport_2014.forEach(d => d.HINCP == ' ' ? '' : d.ADJHH = (d.ADJINC/1000000)*d.HINCP);

''

''

In [35]:
// is there a way to do this without relying on side effects?
function multiply (row, income_col, weight_col, puma_col, tar) {
    for (let i = 0; i < +row[weight_col]; i++) {
        tar.push(row[income_col]);
    }
}

function explode_weights (df, income_col, weight_col, puma_col) {
    /*
    Make a list of HHINCOME for calculating
    a quantile where each household's HHINCOME
    is repeated in the series n times, where
    n == weight_col
    */
    tar = []
    df.forEach(d => multiply(d,income_col,weight_col,puma_col,tar));
    return tar;
}

exploded_bridgeport = explode_weights(bridgeport_2014,"ADJHH","WGTP","PUMA");

''

''

In [36]:
simpleStats.quantile(exploded_bridgeport,0.95);

558969.9774999999

# Hallelujah!

In [37]:
acs_16 = dsv.csvParse(fs.readFileSync('hh_16.csv', 'latin1'));
mable = dsv.csvParse(fs.readFileSync('geocorr14_2014.csv', 'latin1')).slice(1);
// is the mismatch between vintages here a problem?

top_coded = acs_16.filter(d => !d.HD01_VD06)
bottom_coded = acs_16.filter(d => !d.HD01_VD02)

top_coded.forEach(d => d.States = d['GEO.display-label'].split(',')[1].split()[0].split('-'))
// not sure this is doing what we want it to
top_coded.map(d => d.States)

[ [ ' California' ],
  [ ' MA', 'NH Metro Area' ],
  [ ' CO Metro Area' ],
  [ ' CT Metro Area' ],
  [ ' CO Micro Area' ],
  [ ' WY', 'ID Micro Area' ],
  [ ' NM Micro Area' ],
  [ ' CA Metro Area' ],
  [ ' TX Metro Area' ],
  [ ' CA Metro Area' ],
  [ ' FL Metro Area' ],
  [ ' NY', 'NJ', 'PA Metro Area' ],
  [ ' CA Metro Area' ],
  [ ' CA Metro Area' ],
  [ ' CA Metro Area' ],
  [ ' CA Metro Area' ],
  [ ' UT Micro Area' ],
  [ ' NJ Metro Area' ],
  [ ' MA Micro Area' ],
  [ ' DC', 'VA', 'MD', 'WV Metro Area' ],
  [ ' ND Micro Area' ] ]

In [38]:
function make_stpuma(st, puma) {
    return st + ('00000'+puma).slice(-5);
}
make_stpuma(9,102)

'900102'

In [39]:
ct_census.forEach(d => d.stpuma = make_stpuma(d.ST,d.PUMA));

In [40]:
mable.forEach(d => d.stpuma = make_stpuma(d.state, d.puma12))
mable.slice(0,5)

[ { state: '01',
    puma12: '00100',
    cbsa: ' ',
    stab: 'AL',
    cbsaname15: ' ',
    PUMAname: 'Lauderdale, Colbert, Franklin & Marion (Northeast) Counties',
    pop14: '39326.125',
    afact: '0.21 ',
    stpuma: '0100100' },
  { state: '01',
    puma12: '00100',
    cbsa: '22520',
    stab: 'AL',
    cbsaname15: 'Florence-Muscle Shoals, AL (Metro)',
    PUMAname: 'Lauderdale, Colbert, Franklin & Marion (Northeast) Counties',
    pop14: '147639',
    afact: '0.79 ',
    stpuma: '0100100' },
  { state: '01',
    puma12: '00200',
    cbsa: '26620',
    stab: 'AL',
    cbsaname15: 'Huntsville, AL (Metro)',
    PUMAname: 'Limestone & Madison (Outer) Counties--Huntsville City (Far West & Southwest)',
    pop14: '183944.849',
    afact: '1 ',
    stpuma: '0100200' },
  { state: '01',
    puma12: '00301',
    cbsa: '26620',
    stab: 'AL',
    cbsaname15: 'Huntsville, AL (Metro)',
    PUMAname: 'Huntsville (North) & Madison (East) Cities',
    pop14: '124425.297',
    afact: '1 ',
 

In [41]:
all_pumas = [];

function get_pumas(geo_id, geo_corr, rel) {
    cbsa = geo_corr.filter(d => +d.cbsa == +geo_id);
    if (cbsa.length == 0) {
        console.log(geo_id);
    }
    target = {};
    for (index in cbsa) {
        row = cbsa[index];
        
        target[+row.stpuma] = parseFloat(row.afact);
        rel.push(+row.stpuma);
    }
    return target;
}

top_coded.forEach(d => d.PUMAS = get_pumas(d['GEO.id2'], mable, all_pumas));

0640354


In [42]:
state_fips = _(mable).groupBy('stab').mapValues(d => _.maxBy(d,'state')).value();
state_fips['CA']['state']

'06'

In [43]:
/*
import zipfile,requests,io,os

def fetch_census(x,direct):
    zip_path = "https://www2.census.gov/programs-surveys/acs/data/pums/2016/1-Year/csv_h"+x+".zip"
    r = requests.get(zip_path, stream=True)
    zip_ref = zipfile.ZipFile(io.BytesIO(r.content))
    insert = ""
    if direct is not None:
        insert = direct+'/'
    dir_path = "too_big/"+insert+"2016/csv_h"+x+"/"
    zip_ref.extractall(dir_path)
    os.remove("too_big/"+insert+"2016/csv_h"+x+"/ACS2016_PUMS_README.pdf")

def download_unzip_census(ss_s,direct=None):
    ss_s.apply(lambda x: fetch_census(x.lower(),direct))
    
def make_path(year,state,direct):
    insert = ""
    if direct is not None:
        insert = direct +'/'
    return ("too_big/"+insert+str(year)+"/csv_h"+state+"/ss16h"+state+".csv")
    
def get_census_pums(ss_s,model,direct=None):
    tar = pandas.DataFrame(columns=model.columns)
    for index, s in ss_s.iteritems():
        print(s)
        file = make_path(2016,s.lower(),direct)
        new = pandas.read_csv(file) ## Grab for every topcoded state
        statefip = state_fips.loc[s.upper()]["state"]
        new["stpuma"] = new.apply(lambda x: make_stpuma(x["ST"],x["PUMA"]),axis=1)
        tar = pandas.concat([tar,new])
    return tar
*/

In [44]:
pop_16 = dsv.csvParse(fs.readFileSync('ACS_16_1YR_B01003.csv', 'latin1')); // total population by metro area

top_coded = top_coded.filter(d =>
    (d['GEO.display-label'].indexOf('Metro') !== -1) && // filter to metro areas only
    (d['GEO.display-label'].indexOf(' PR ') !== -1) && // this might do nothing?
    (+pop_16.find(d2 => d['GEO.id2'] == d2['GEO.id2']).HD01_VD01 >= 500000) // population at or over 500,000
)

top_coded.length

0

In [45]:
function pums_dataset (tc, exploded_pickle, is_rel, states_dir=null) {
    s = pandas.Series(tc["States"].sum()).drop_duplicates().sort_values()
    s = s[s!="California"]
    sls = s.tolist()
    print(s)
    if not (os.path.isfile(make_path(2016,sls[0],states_dir))):
        download_unzip_census(s,states_dir)
    census_pums = get_census_pums(s,ct_census,states_dir)
    census_pums["relevant"] = census_pums["stpuma"].apply(lambda x: x in is_rel)
    relevant_census = census_pums[census_pums["relevant"]]
    relevant_census["HHINC"] = pandas.to_numeric(relevant_census["HINCP"],errors="coerce")
    exploded_census = explode_weights(relevant_census,"HHINC","WGTP","stpuma")
    write_obj(exploded_census,exploded_pickle)
    return exploded_census
}
    
exploded_census = pums_dataset(top_coded,"too_big/exploded_census.pickle",all_pumas)
exploded_census = pandas.DataFrame(exploded_census)

SyntaxError: Unexpected identifier